# Prediction of Sales Transactions


### Import all the necessary libraries and packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn import datasets
import ast
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

### Load the data from the data source and analyze it

In [2]:
df = pd.read_csv('./sales_data.csv')
df.head()

,user_id,user_id_type,cart_id,date_modified,currency,unit_price,net_price,discountpercent,recurring_unit_price,recurring_net_price,recurring_discountpercent,quantity,product_system_id,deal_type,opportunity_region,opportunity_budget,contract_term,total_contract_value,deal_category
0,525,8477949227256649820,525-1,02/22/2017 03:30:01.050000,GBP,0.0,0.0,0.0,0.000000,0.000000,0.0,78.0,1789161784535801413,Renewal,UK&I,"""Less than 1",0.0,12,-1575416002466453954
1,525,8477949227256649820,525-1,02/22/2017 03:30:01.050000,GBP,0.0,0.0,0.0,6.923077,6.923077,0.0,78.0,-3388033280723878524,Renewal,UK&I,"""Less than 1",0.0,12,-1575416002466453954
2,525,8477949227256649820,525-1,02/22/2017 03:30:01.050000,GBP,0.0,0.0,0.0,0.000000,0.000000,0.0,8.0,-9130972624394922538,Renewal,UK&I,"""Less than 1",0.0,12,-1575416002466453954
3,525,8477949227256649820,525-1,02/22/2017 03:30:01.050000,GBP,0.0,0.0,0.0,187.500000,187.500000,0.0,8.0,2739638421988212911,Renewal,UK&I,"""Less than 1",0.0,12,-1575416002466453954
4,525,8477949227256649820,525-1,02/22/2017 03:30:01.050000,GBP,0.0,0.0,0.0,0.000000,0.000000,0.0,8.0,-6185560453609870674,Renewal,UK&I,"""Less than 1",0.0,12,-1575416002466453954


### Count the number of NaN or 0's for each column in DataFrame

In [3]:
df.isnull().sum()

user_id                      0
user_id_type                 0
cart_id                      0
date_modified                0
currency                     0
unit_price                   0
net_price                    0
discountpercent              0
recurring_unit_price         0
recurring_net_price          0
recurring_discountpercent    0
quantity                     0
product_system_id            0
deal_type                    0
opportunity_region           0
opportunity_budget           0
contract_term                0
total_contract_value         0
deal_category                0
dtype: int64

### Compute the length of unique elemenst for each column

In [4]:
for col in df:
    print(col, len(df[col].unique()))

user_id 557
user_id_type 21
cart_id 23884
date_modified 23884
currency 7
unit_price 3801
net_price 4687
discountpercent 2315
recurring_unit_price 3644
recurring_net_price 3663
recurring_discountpercent 426
quantity 944
product_system_id 275
deal_type 4
opportunity_region 4
opportunity_budget 103
contract_term 90
total_contract_value 364
deal_category 12


## Data Cleaning and Feature Selection

### Omitting the user_id, user_id_type and cart_id from the data as it wont be necessary to predict the discounts

In [ ]:
df_new= df.iloc[:,3:]
df_new.head()

,date_modified,currency,unit_price,net_price,discountpercent,recurring_unit_price,recurring_net_price,recurring_discountpercent,quantity,product_system_id,deal_type,opportunity_region,opportunity_budget,contract_term,total_contract_value,deal_category
0,02/22/2017 03:30:01.050000,GBP,0.0,0.0,0.0,0.000000,0.000000,0.0,78.0,1789161784535801413,Renewal,UK&I,"""Less than 1",0.0,12,-1575416002466453954
1,02/22/2017 03:30:01.050000,GBP,0.0,0.0,0.0,6.923077,6.923077,0.0,78.0,-3388033280723878524,Renewal,UK&I,"""Less than 1",0.0,12,-1575416002466453954
2,02/22/2017 03:30:01.050000,GBP,0.0,0.0,0.0,0.000000,0.000000,0.0,8.0,-9130972624394922538,Renewal,UK&I,"""Less than 1",0.0,12,-1575416002466453954
3,02/22/2017 03:30:01.050000,GBP,0.0,0.0,0.0,187.500000,187.500000,0.0,8.0,2739638421988212911,Renewal,UK&I,"""Less than 1",0.0,12,-1575416002466453954
4,02/22/2017 03:30:01.050000,GBP,0.0,0.0,0.0,0.000000,0.000000,0.0,8.0,-6185560453609870674,Renewal,UK&I,"""Less than 1",0.0,12,-1575416002466453954


### Fetch only the day from date_modified column

In [ ]:
df_new['date'] = '02/22/2017 03:30:01.050000'
df_new['days'] =  pd.DatetimeIndex(df_new['date_modified']) - pd.DatetimeIndex(df_new['date'])
df_new['days'] = df_new['days'].dt.days

### Identify the columns that require Label Encoding i.e df_cat

In [ ]:
df_cat = df_new[['currency','product_system_id','deal_type','opportunity_region','opportunity_budget','deal_category']]
df_cont = df_new[['unit_price','net_price','discountpercent','recurring_unit_price','recurring_net_price','recurring_discountpercent','quantity','contract_term','total_contract_value']]

### Apply the Label Encoding so that all the categorical data are represented in numerical fashion

In [ ]:
df_cat = df_cat.apply(LabelEncoder().fit_transform)

### Concatenate the encoded data with already existed data

In [ ]:
df_processed = pd.concat([df_cat, df_cont, df_new['days']], axis=1)

In [ ]:
df_processed['total_contract_value'] = df_cont['total_contract_value'].apply(lambda x : ast.literal_eval(x.replace("\"", "")))

### Analyzing the feature selection by plotting seaborn heatmap

In [ ]:
plt.figure(figsize =(15,15))
sns.heatmap(df_processed.corr(method='pearson'), 
            xticklabels=df_processed.columns,
            yticklabels=df_processed.columns, annot=True)

### Using seaborn pearson heatmap plot, the columns 'opportunity_budget' and 'contract_term' seems to be less correlated with the target 'discountpercent'. So drop the above columns for accuracy and efficiency.

In [ ]:
X1 = df_processed.drop(['discountpercent','recurring_discountpercent','opportunity_budget','contract_term'],axis=1)
Y1 = df_processed[['discountpercent','recurring_discountpercent']]

### Split the data into training data and test data 

In [ ]:
trainX, testX, trainY, testY = train_test_split(X1, Y1, test_size=0.3)

## KFold Cross Validation - to decide which model performs better and also to prevent overfitting

### Pipeline method is used for KFold Cross Validation based on negative mean squared error. 
### The model performace is judged based on standard deviation and mean

In [ ]:
pipelines = []
pipelines.append(('ScaledLR', Pipeline([('Scaler', StandardScaler()),('LR',LinearRegression())])))
pipelines.append(('ScaledLASSO', Pipeline([('Scaler', StandardScaler()),('LASSO', Lasso())])))
pipelines.append(('ScaledEN', Pipeline([('Scaler', StandardScaler()),('EN', ElasticNet())])))
pipelines.append(('ScaledLG', Pipeline([('Scaler', StandardScaler()),('LG',LogisticRegression())])))
pipelines.append(('ScaledKNN', Pipeline([('Scaler', StandardScaler()),('KNN', KNeighborsRegressor())])))
pipelines.append(('ScaledCART', Pipeline([('Scaler', StandardScaler()),('CART', DecisionTreeRegressor())])))
pipelines.append(('ScaledRFRT', Pipeline([('Scaler', StandardScaler()),('RFRT', RandomForestRegressor())])))
pipelines.append(('ScaledGBM', Pipeline([('Scaler', StandardScaler()),('GBM', GradientBoostingRegressor())])))

results = []
names = []
for name, model in pipelines:
    kfold = KFold(n_splits=10, random_state=0)
    cv_results = cross_val_score(model, trainX, (trainY.iloc[:,0]).astype(int), cv=kfold, scoring='neg_mean_squared_error')
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

### The outputs of each model are as below - 
#### Linear Regression = -293.802154 (11.930265)
#### Lasso Regression = -301.454122 (14.369864)
#### Elastic Net Regression = -310.448764 (14.441288)
#### Decision Tree Regression = -11.019956 (2.823359)
#### Random Forest Regression = -7.274902 (2.840953)
#### Gradient Boosting Regression = -38.063820 (4.331756)

#### => As Random Forest Regression has the lowest value = -7.27 comparatively, hence we go ahead to build the model using Random Forest Regression.

In [ ]:
regrf = RandomForestRegressor(n_estimators=200, random_state=0, max_features="sqrt")
RF = regrf.fit(trainX, (trainY.iloc[:,0]).astype(int))

### Calculated Mean Squared Error = 13.81

In [ ]:
print(mean_squared_error((testY.iloc[:,0]).astype(int), regrf.predict(testX)))

### Calculated Mean Absolute Error = 0.4648

In [ ]:
print(mean_absolute_error((testY.iloc[:,0]).astype(int), regrf.predict(testX)))

### Calculated R-Squared Value = 0.962

In [ ]:
print(r2_score((testY.iloc[:,0]).astype(int), regrf.predict(testX)))

## As target 'recurring_discountpercent' highly depends on the previously predicted 'discountpercent', hence it is passed to train the next model to predict 'recurring_discountpercent'

In [ ]:
testX['predicted_discount'] = regrf.predict(testX)
X2 = testX
Y2 = testY.iloc[:,1]

In [ ]:
trainX2, testX2, trainY2, testY2 = train_test_split(X2, Y2, test_size=0.3)

## Random Forest Regression is used to predict 'recurring_discountpercent'

In [ ]:
regrf2=RandomForestRegressor(n_estimators=200, max_depth=None, max_features="sqrt")
RF2 = regrf2.fit(trainX2, trainY2.astype(int))

### Calculated Mean Squared Error = 9248.86

In [ ]:
print (mean_squared_error(testY2.astype(int), regrf2.predict(testX2)))

### Calculated Mean Absolute Error = 1.7842

In [ ]:
print(mean_absolute_error(testY2.astype(int), regrf2.predict(testX2)))

### Calculated R-Squared Value = 0.717

In [ ]:
print(r2_score(testY2.astype(int), regrf2.predict(testX2)))

## Future Directions

### Use other column values such as 'recurring_net_price', 'net_price', etc to predict 'recurring_discountpercent'
### Further can analyze the data and accordingly can select features which help the accuracy, also keeping in mind  about the overfitting

